0. 일단 [composition] 으로 전체적으로 바꿔야 한다. 

1. 이 때 df.TAGDESC_10 을 우선 TAGDESC_11 로 카피한 다음 
2. 매칭되면 TAGDESC_11 을 바꿔주고, 
3. 이 때 df_comp 에 매칭된 단어 카운트를 써주는 것도 결과 분석 시 도움이 되겠군.
4. 다음 df 를 lambda 로 접근하여 
  * 일단 단어를 쪼개 리스트에 넣고 
  * 하나하나 df_dic 에서 찾아서 바꾼다.
  * 이 때 발견되지 못한 단어는 df_undef.단어 에 추가
  
## Q.
- 영어로만 이루어진 짧은 단어, 예를 들면 'OK' 도 단어사전에 있어야 하나?

In [1]:
import pandas as pd
import numpy as np
import re

In [80]:
df = pd.read_csv('./tagdesc_index.csv', encoding='euc-kr')
df

,NO,태그설명,TAGDESC_10
0,1,#1 ALARM,1 ALARM
1,2,NaN,정상동작 OK
2,3,NaN,값 농도1
3,4,NaN,스크린 파손유무 08
4,5,NaN,전기다리미 BI METAL


In [30]:
# 단어+단어
df_comp = pd.read_csv('./단어PLUS단어.csv', encoding='euc-kr')
df_comp['matched_cnt'] = np.nan
df_comp

,CODE_NAME,COMM_CODE,matched_cnt
0,정상동작,정상 동작,NaN
1,농도1,농도 1,NaN
2,파손유무,파손 유무,NaN
3,BI METAL,BIMETAL,NaN
4,HELLOWORLD,HELLO WORLD,NaN


In [184]:
# 단어사전
df_dic = pd.read_csv('./단어사전.csv', encoding='euc-kr')
df_dic

,No,단어,단어 길이,CODE,CODE 길이
0,11417.0,천장,NaN,CEIL,NaN
1,NaN,정상,NaN,OK,NaN
2,NaN,OK,NaN,OK,NaN
3,NaN,동작,NaN,OPER,NaN


# 1. 이 때 df.TAGDESC_10 을 우선 TAGDESC_11 로 카피한 다음

In [81]:
df['TAGDESC_11'] = df['TAGDESC_10']
df

,NO,태그설명,TAGDESC_10,TAGDESC_11
0,1,#1 ALARM,1 ALARM,1 ALARM
1,2,NaN,정상동작 OK,정상동작 OK
2,3,NaN,값 농도1,값 농도1
3,4,NaN,스크린 파손유무 08,스크린 파손유무 08
4,5,NaN,전기다리미 BI METAL,전기다리미 BI METAL


# 2. 매칭되면 TAGDESC_11 을 바꿔주고

In [65]:
def parse_comp_on_matched(row, key, val):
    idx = row['NO']
    desc = row['TAGDESC_11']
    result = re.sub(key, val, desc)
    df.loc[df['NO'] == idx, 'TAGDESC_11'] = result
    pass

In [66]:
def parse_comp(row):
    key = row['CODE_NAME']
    val = row['COMM_CODE']
    df_m = df[df['TAGDESC_10'].str.contains(r'^.*' + key + '.*')]
    # 이제 매칭된 것마다 값을 바꿔보자.
    df_m.apply(lambda x: parse_comp_on_matched(x, key, val), axis=1)
    row['matched_cnt'] = len(df_m)
    return row

In [82]:
df_comp = df_comp.apply(lambda x: parse_comp(x), axis=1)
df_comp

,CODE_NAME,COMM_CODE,matched_cnt
0,정상동작,정상 동작,1
1,농도1,농도 1,1
2,파손유무,파손 유무,1
3,BI METAL,BIMETAL,1
4,HELLOWORLD,HELLO WORLD,0


In [83]:
df

,NO,태그설명,TAGDESC_10,TAGDESC_11
0,1,#1 ALARM,1 ALARM,1 ALARM
1,2,NaN,정상동작 OK,정상 동작 OK
2,3,NaN,값 농도1,값 농도 1
3,4,NaN,스크린 파손유무 08,스크린 파손 유무 08
4,5,NaN,전기다리미 BI METAL,전기다리미 BIMETAL


# 4. 다음 df 를 lambda 로 접근하여

In [181]:
# Explicit is better than implicit
df_undef = pd.DataFrame(data=None, columns=df_dic.columns)
df_undef

,No,단어,단어 길이,CODE,CODE 길이


In [84]:
# 초기화
df['TAGDESC_20'] = np.nan
df

,NO,태그설명,TAGDESC_10,TAGDESC_11,TAGDESC_20
0,1,#1 ALARM,1 ALARM,1 ALARM,NaN
1,2,NaN,정상동작 OK,정상 동작 OK,NaN
2,3,NaN,값 농도1,값 농도 1,NaN
3,4,NaN,스크린 파손유무 08,스크린 파손 유무 08,NaN
4,5,NaN,전기다리미 BI METAL,전기다리미 BIMETAL,NaN


In [162]:
# 중복 체크 후 추가.
def on_add_word(token):
    global df_undef
    df_m = df_undef[df_undef['단어'] == token]
    if len(df_m) == 0:
        df_undef = df_undef.append({'단어': token}, ignore_index=True)

In [176]:
"""
일단 단어를 쪼개 리스트에 넣고
하나하나 c 에서 찾아서 바꾼다.
이 때 발견되지 df_di못한 단어는 df_undef.단어 에 추가
"""
def parse_desc(row):
    desc = row['TAGDESC_11']
    tokens = desc.split()
    result = [''] * len(tokens)
    #
    for i in range(len(tokens)):
        token = tokens[i]
        # 모두 숫자라면
        if re.match(r'^\d+$', token):
            result[i] = token
            continue
        df_m = df_dic.loc[df_dic['단어'] == token]
        matched_len = len(df_m)
        if matched_len == 0:
            on_add_word(token)
            result[i] = 'UNDEF'
        elif matched_len == 1:
            s = df_m.iloc[0]
            code = s['CODE']
            result[i] = code
        else:
            s = df_m.iloc[0]
            code = s['CODE']
            raise Exception('[ERROR] 단어사전에 중복된 단어가 있습니다. => <{}>'.format(token))
    row['TAGDESC_20'] = '_'.join(result)
    return row

In [189]:
df_dic = pd.read_csv('./단어사전.csv', encoding='euc-kr')
df_undef = pd.DataFrame(data=None, columns=df_dic.columns)
df.apply(lambda x: parse_desc(x), axis=1)

,NO,태그설명,TAGDESC_10,TAGDESC_11,TAGDESC_20
0,1,#1 ALARM,1 ALARM,1 ALARM,1_UNDEF
1,2,NaN,정상동작 OK,정상 동작 OK,OK_OPER_OK
2,3,NaN,값 농도1,값 농도 1,VAL_UNDEF_1
3,4,NaN,스크린 파손유무 08,스크린 파손 유무 08,UNDEF_UNDEF_UNDEF_08
4,5,NaN,전기다리미 BI METAL,전기다리미 BIMETAL,UNDEF_BIMETAL


In [190]:
df_undef

,No,단어,단어 길이,CODE,CODE 길이
0,NaN,ALARM,NaN,NaN,NaN
1,NaN,농도,NaN,NaN,NaN
2,NaN,스크린,NaN,NaN,NaN
3,NaN,파손,NaN,NaN,NaN
4,NaN,유무,NaN,NaN,NaN
5,NaN,전기다리미,NaN,NaN,NaN
